#  Определение предельных динамических характеристик позиционера 

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import ipywidgets as widgets

### Максимальное ускорение

$$
\beta_{max} = \frac{N_{max} - N_{тр}}{J} = \frac{k_M \cdot I_{max} - N_{тр}}{J}
$$

* $\beta_{max}$ - максимальное угловое ускорение
* $N_{тр}$ - момент силы сухого трения
* $k_M$ - коэффициент момента - коэффициент пересчёта тока в момент силы \[Н*м / А\], берётся из документации на двигатель
* $I_{max}$ - максимальный среднеквадратичный ток
* $J$ - момент инерции ротора относительно оси вращения

### Максимальный рывок

$$
U_{max} = L\left(\frac{dI}{dt}\right)_{max}
$$

$$
\gamma_{max} = \left(\frac{d\beta}{dt}\right)_{max} = \frac{k_M}{J} \left(\frac{dI}{dt}\right)_{max} = \frac{k_M}{J} \frac{U_{max}}{L}
$$

* $L$ - индуктивность обмоток двигателя, берётся из документации на двигатель
* $U_{max}$ - максимальное напряжение
* $\gamma_{max}$ - максимальный угловой рывок (производная от углового ускорения)

### Ограничение на скорость из учёта обратной индукции

Вводим требование:

$$
\frac{U_{EMF}}{U_{max}} < 30 \%
$$

$$
\omega < \frac{3}{10} \frac{U_{max}}{k_{EMF}}
$$

* $U_{EMF}$ - обратная ЭДС
* $U_{MAX}$ - максимальное напряжение
* $k_{EMF}$ - коэффициент обратной ЭДС (противоЭДС) \[В / (тыс. об/мин)\], берётся из документации на двигатель
* $\omega$ - скорость

## Калькулятор предельных динамических характеристик

In [5]:
n_tr_input = widgets.BoundedFloatText(
    value=0,
    min=0,
    max=100,
    step=0.1,
    description="$N_{тр}$, Н",
    tooltip="Момент силы сухого трения в Ньютонах * метр"
)
k_m_input = widgets.BoundedFloatText(
    value=12.9,
    min=0.001,
    max=100000,
    step=0.1,
    description="$k_{M}$, Н*м/А",
    tooltip="Коэффициент момента (берётся из документации на двигатель)"
)
i_max_input = widgets.BoundedFloatText(
    value=1,
    min=0.01,
    max=100,
    step=0.1,
    description="$I_{max}$, A",
    tooltip="Максимальный ток"
)
j_input = widgets.BoundedFloatText(
    value=2.2,
    min=0.001,
    max=10000,
    step=0.1,
    description="$J$, кг*м^2",
    tooltip="Момент инерции ротора относително оси вращения"
)
u_max_input = widgets.BoundedFloatText(
    value=36,
    min=0.01,
    max=1000,
    step=0.1,
    description="$U_{max}$, В",
    tooltip="Максимальное напряжение"
)
inductance_input = widgets.BoundedFloatText(
    value=0.00124,
    min=0.0000001,
    max=0.1,
    step=0.1,
    description="$L$, Гн",
    tooltip="Индуктивность обмоток двигателя, берётся из документации на двигатель"
)
k_emf_input = widgets.BoundedFloatText(
    value=15,
    min=0.0000001,
    max=100000,
    step=0.1,
    description="$k_{emf}$, В/(тыс. об/мин)",
    tooltip="Коэффициент обратной ЭДС, берётся из документации на двигатель"
)

calculate_dynamic_limits_button = widgets.Button(
    description="Рассчитать"
)

dl_output = widgets.Output()

def calculate_dynamic_limits(b):
    with dl_output:  
        dl_output.clear_output()
        n_tr = float(n_tr_input.value)
        k_m = float(k_m_input.value)
        i_max = float(i_max_input.value)
        j = float(j_input.value)
        u_max = float(u_max_input.value)
        inductance = float(inductance_input.value)
        k_emf = float(k_emf_input.value)
        
        beta_max = (k_m * i_max - n_tr) / j
        j_max = k_m * u_max / (j * inductance)
        v_emf_max = 0.3 * u_max / k_emf
        
        print("Максимальное ускорение: {} рад/с²".format(np.round(beta_max, 3)))
        print("Максимальный рывок: {} рад/с³".format(np.round(j_max, 0)))
        print("Ограничение на скорость из учёта обратной индукции: {} тыс. об/мин".format(np.round(v_emf_max, 6)))

calculate_dynamic_limits_button.on_click(calculate_dynamic_limits)

display(
    n_tr_input,
    k_m_input,
    i_max_input,
    j_input,
    u_max_input,
    inductance_input,
    k_emf_input,
    calculate_dynamic_limits_button,
    dl_output
)

BoundedFloatText(value=0.0, description='$N_{тр}$, Н', step=0.1)

BoundedFloatText(value=12.9, description='$k_{M}$, Н*м/А', max=100000.0, min=0.001, step=0.1)

BoundedFloatText(value=1.0, description='$I_{max}$, A', min=0.01, step=0.1)

BoundedFloatText(value=2.2, description='$J$, кг*м^2', max=10000.0, min=0.001, step=0.1)

BoundedFloatText(value=36.0, description='$U_{max}$, В', max=1000.0, min=0.01, step=0.1)

BoundedFloatText(value=0.00124, description='$L$, Гн', max=0.1, min=1e-07, step=0.1)

BoundedFloatText(value=15.0, description='$k_{emf}$, В/(тыс. об/мин)', max=100000.0, min=1e-07, step=0.1)

Button(description='Рассчитать', style=ButtonStyle())

Output()

## Подбор оптимальной тестовой трапеции

$$
x = x_0 + v_0 t + a_0\frac{t^2}{2} + j\frac{t^3}{6} \\
v = v_0 + a_0 t + j\frac{t^2}{2} \\
a = a_0 + jt
$$

Перемещение на расстояние $x_{target}$ в общем случае включает в себя следующие фазы:

* Рывок (увеличение / уменьшение ускорения) – 4 сегмента
* Равноускоренное / равнозамедленное движение – 2 сегмента
* Движение с постоянной скоростью – 1 сегмент

### Рывок (набор ускорения)

Движение в любом случае начинается с рывка.

$$
a(t) = j t \\
v(t) = j \frac{t^2}{2}\\
x(t) = j \frac{t^3}{6}
$$

Рывок заканчивается в тот момент, когда наступает первое (по времени) из следующих событий:
* Достигнуто максимальное ускорение $a(t_{jerk}) = a_{max}$ – ускоряться дальше нельзя.
* Скорость стала равна половине максимальной $v(t_{jerk}) = \frac{1}{2}v_{max}$ – пора делать обратный рывок, чтобы прекратить ускорение к достижению $v=v_max$.
* Пройдена четверть пути $x(t_{jerk}) = \frac{1}{12}x_{target}$ – пора уменьшать ускорение и сразу начинать торможение, иначе не успеем затормозить до достижения $x = x_{target}$.


$$
t_{jerk} = min\left(\frac{a_{max}}{j} ;\quad \sqrt{\frac{v_{max}}{j}};\quad \sqrt[3]{\frac{x_{max}}{2j}} \right) \\
$$

> #### Почему в третьем условии $\frac{1}{12}x_{target}$?
>
> Рассмотрим перемещение состоящее только из двух положительных рывков и двух отрицательных рывков. 
> Точнее только из соображений симметрии будем рассматривать только половину за которую мы должны пройти $\frac{1}{2}x_{target}$.
>
> Положительный рывок уже рассмотрен выше. Смотрим координату после отрицательного рывка.
>
> После отрицательного рывка ускорение должно быть равно нулю: 
 
$$
0 = a_{jerk} - j t_{jerk_2} \Rightarrow t_{jerk_2} = \frac{a_{jerk}}{j} = \frac{j t_{jerk_1}}{j} = t_{jerk_1} = t_{jerk}
$$

> Пройденное расстояние после отрицательного рывка должно быть $\frac{x_{target}}{2}$:

$$
\frac{1}{2}x_{target} = x_{jerk} + v_{jerk} t_{jerk} + a_{jerk} \frac{t^2_{jerk}}{2} - j \frac{t^3_{jerk}}{6} = 
j \frac{t^3_{jerk}}{6} + j \frac{t^3_{jerk}}{2} + j \frac{t^3_{jerk}}{2} - jj\frac{t^3_{jerk}}{6} = j t^3_{jerk} \\
t_{jerk} = \sqrt[3]{\frac{x_{target}}{2j}} \\
x_{jerk} = \frac{x_{target}}{12}
$$

### Равноускоренное движение

Если после первого рывка обратный рывок не начинается сразу, то далее следует сегмент равноускоренного движения.

На входе:

$$
a_{jerk} = j t_{jerk}\\
v_{jerk} = j \frac{t^2_{jerk}}{2}\\
x_{jerk} = j \frac{t^3_{jerk}}{6}
$$

В процессе движения:

$$
a(t) = a_{jerk} = j t_{jerk}\\
v(t) = v_{jerk} + a_{jerk} t = j t^2_{jerk} \left(\frac{1}{2} + \frac{t}{t_{jerk}} \right)\\
x(t) = x_{jerk} + v_{jerk} t + a_{jerk} \frac{t^2}{2} = j t^3_{jerk} \left(\frac{1}{6} + \frac{1}{2}\frac{t}{t_{jerk}} + \frac{1}{2}\left(\frac{t}{t_{jerk}}\right)^2\right)
$$


Равноускоренное движение завершается при возникновении первого из следующих событий:
* Достигнута скорость $v(t) = v_{max} – 2v_{jerk}$ - пора начинать завершать ускорение, иначе скорость может превысить $v_{max}$.
* Пройден такой путь  $x(t)$, что сразу после завершения обратного рывка, начатого в данной точке, будет пройден путь $\frac{1}{2}x_{target}$.


$$
t_a = min\left(t_{jerk}\left(\frac{v_{max}}{jt^2_{jerk}} - 1 \right);\;  t_{jerk} \frac{1}{2}\left(\sqrt{1 + 4\frac{x_{target}}{jt^3_{jerk}}} - 3 \right)\right)
$$

> #### Как получилось критическое время для достижения максимальной скорости

$$
v_{max} - v_{jerk} = v_{jerk} + a_{jerk}t \\
t = \frac{v_{max}}{a_{jerk}} - 2\frac{v_{jerk}}{a_{jerk}} = \frac{v_{max}}{jt_{jerk}} - 2\frac{\frac{1}{2}jt^2_{jerk}}{jt_{jerk}} = t_{jerk}\left( \frac{v_{max}}{jt^2_{jerk}} - 1\right)
$$

> #### Как получилось критическое время для прохождения $\frac{1}{2}x_{target}$
>
> После завершения участка равноускоренного движения:
 
$$
v(t) = j t^2_{jerk} \left(\frac{1}{2} + \frac{t}{t_{jerk}} \right) \\
x(t) = j t^3_{jerk} \left(\frac{1}{6} + \frac{1}{2}\frac{t}{t_{jerk}} + \frac{1}{2}\left(\frac{t}{t_{jerk}}\right)^2\right)
$$

> Далее следует отрицательный рывок, в ходе которого ускорение $a_{jerk}$ должно уменьшиться до нуля: $0 = a_{jerk} - jt_{jerk_2}$. Следовательно: $t_{jerk_2} = \frac{a_{jerk}}{j} = t_{jerk_1} = t_{jerk}$.
>
> Пройденное расстояние после этого рывка должно быть $\frac{1}{2}x_{target}$:

$$
\frac{1}{2}x_{target} = x_a + v_a t_{jerk} + a_{jerk}\frac{t_{jerk}}{2} - j\frac{t^3_{jerk}}{6} = \\
= j t^3_{jerk} \left(\frac{1}{6} + \frac{1}{2}\frac{t}{t_{jerk}} + \frac{1}{2}\left(\frac{t}{t_{jerk}}\right)^2\right)
+ j t^2_{jerk} \left(\frac{1}{2} + \frac{t}{t_{jerk}} \right) t_{jerk} + jt_{jerk}\frac{t_{jerk}}{2} - j\frac{t^3_{jerk}}{6} = \\
= j t^3_{jerk}\frac{1}{2}  \left(\left(\frac{t}{t_{jerk}}\right)^2 + 3\frac{t}{t_{jerk}} + 2 \right)
$$

> Время равноускоренного движение $t_a$ является решением квадратного уравнения:

$$
j t^3_{jerk}\frac{1}{2}  \left(\left(\frac{t}{t_{jerk}}\right)^2 + 3\frac{t}{t_{jerk}} + 2  - \frac{x_{target}}{j t^3_{jerk}}\right) = 0\\
t = t_{jerk}\frac{1}{2} \left(\sqrt{1 + 4\frac{x_{target}}{j t^3_{jerk}}} -3\right)
$$

### Отрицательный рывок

Длительность отрицательного рывка равна длительности первого рывка $t_{jerk}$, которая была посчитана ранее (доказано выше).

На входе:

$$
a_{target} = a_{jerk} = j t_{jerk}\\
v_a = v_{jerk} + a_{jerk} t_a = j t^2_{jerk} \left(\frac{1}{2} + \frac{t_a}{t_{jerk}} \right)\\
x_a = x_{jerk} + v_{jerk} t_a + a_{jerk} \frac{t_a^2}{2} = j t^3_{jerk} \left(\frac{1}{6} + \frac{1}{2}\frac{t_a}{t_{jerk}} + \frac{1}{2}\left(\frac{t_a}{t_{jerk}}\right)^2\right)
$$

В процессе движения:

$$
a(t) = a_{jerk} - j t = j(t_{jerk} - t)\\
v(t) = v_a + a_{jerk} t - j \frac{t^2}{2} = jt^2_{jerk} \left( \frac{1}{2} + \frac{t_a}{t_{jerk}} + \frac{t}{t_{jerk}} - \frac{1}{2}\left(\frac{t}{t_{jerk}}\right)^2 \right) \\
x(t) = x_a + v_a t + a_{jerk} \frac{t^2}{2} - j\frac{t^3}{6} = j t^3_{jerk} \left(\frac{1}{6} + \frac{1}{2}\frac{t_a}{t_{jerk}} + \frac{1}{2}\left(\frac{t_a}{t_{jerk}}\right)^2 + \frac{1}{2}\frac{t}{t_{jerk}} + \frac{t_a}{t_{jerk}}\frac{t}{t_{jerk}} + \frac{1}{2} \left(\frac{t}{t_{jerk}}\right)^2 - \frac{1}{6} \left(\frac{t}{t_{jerk}}\right)^3 \right)
$$

### Равномерное движение

На входе:

$$
a = 0\\
v_{target} = jt^2_{jerk} \left( \frac{t_a}{t_{jerk}} + 1 \right) \\
x_{speedup} = jt^3_{jerk} \frac{1}{2} \left( \left(\frac{t_a}{t_{jerk}}\right)^2 + 3\frac{t_a}{t_{jerk}} +2  \right)
$$

В процессе движения:

$$
a = 0\\
v = v_{target} = jt^2_{jerk} \left( \frac{t_a}{t_{jerk}} + 1 \right)\\
x(t) = x_{speedup} + v_{target}t = jt^3_{jerk} \frac{1}{2} \left( \left(\frac{t_a}{t_{jerk}}\right)^2 + 3\frac{t_a}{t_{jerk}} +2 + \frac{t_a}{t_{jerk}}\frac{t}{t_{jerk}} + \frac{t}{t_{jerk}} \right) = jt^3_{jerk} \frac{1}{2} \left( \frac{t_a}{t_{jerk}} + 1 \right) \left( \frac{t_a}{t_{jerk}} + + 2 \frac{t}{t_{jerk}} \right)
$$

Полный разгон и полное торможение занимают одно и то же время $t_{a} + 2t_{jerk}$ и одно и то же расстояние $x_{speedup}$. Соответственно, расстояние, которое будет пройдено с фиксированной скоростью: $x_{u} = x_{target} – 2 x_{speedup}$. Зная это расстояние можно вычислить время равномерного движения $t_u$.

$$
t_{u} = \frac{x_{target} - 2x_{speedup}}{v_{target}} = \frac{x_{target}}{jt^2_{jerk}\left( \frac{t_a}{t_{jerk}} + 1\right)} - \left( t_a + 2 t_{jerk}\right)
$$

## Основные формулы ещё раз

$$
t_{jerk} = min\left(\frac{a_{max}}{j} ;\quad \sqrt{\frac{v_{max}}{j}};\quad \sqrt[3]{\frac{x_{max}}{2j}} \right) \\
t_a = min\left(t_{jerk}\left(\frac{v_{max}}{jt^2_{jerk}} - 1 \right);\;  t_{jerk} \frac{1}{2}\left(\sqrt{1 + 4\frac{x_{target}}{jt^3_{jerk}}} - 3 \right)\right) \\
t_{u} = \frac{x_{target}}{jt^2_{jerk}\left( \frac{t_a}{t_{jerk}} + 1\right)} - \left( t_a + 2 t_{jerk}\right) \\
$$


$$
a_{target} = j t_{jerk}\\
v_{target} = jt^2_{jerk} \left( \frac{t_a}{t_{jerk}} + 1 \right)
$$

In [4]:
def calcaulate_timings_and_targets(x_target, v_max, a_max, j):
    t_jerk = np.min((a_max / j, np.sqrt(v_max/j), np.cbrt(x_target / (2*j))))

    t_a = np.min((
        t_jerk * (v_max / (j * t_jerk * t_jerk) - 1),
        t_jerk * 0.5 * (np.sqrt(1 + 4 * x_target / (j * t_jerk * t_jerk * t_jerk)) - 3)
    ))

    t_u = x_target / (j * t_jerk * t_jerk * (t_a / t_jerk + 1)) - (t_a + 2 * t_jerk)

    a_target = j * t_jerk
    v_target = j * t_jerk * t_jerk * (t_a / t_jerk + 1)

    print("Длительность рыка t_jerk: {} с".format(t_jerk))
    print("Длительность равноускоренного движения t_a: {} с".format(t_a))
    print("Длительность равномерного движения t_u: {} с".format(t_u))
    print("Реальное максимальное ускорение a_target: {} м/с²".format(a_target))
    print("Реальная максимальная скорость v_target: {} м/с".format(v_target))
    
    return(t_jerk, t_a, t_u, a_target, v_target)

In [ ]:
def calculate_time_series(t_jerk, t_a, t_u, j):
    h_t = 1 / 10000000  # c

    t_segments = [
        0,                                                     # 0 - positive jerk
        t_jerk,                                                # 1 - positive acceleration
        t_jerk + t_a,                                          # 2 - negative jerk
        t_jerk + t_a + t_jerk,                                 # 3 - uniform motion
        t_jerk + t_a + t_jerk + t_u,                           # 4 - negative jerk
        t_jerk + t_a + t_jerk + t_u + t_jerk,                  # 5 - negative acceleration
        t_jerk + t_a + t_jerk + t_u + t_jerk + t_a,            # 6 - positive jerk
        t_jerk + t_a + t_jerk + t_u + t_jerk + t_a + t_jerk,   # 7 - the end
    ]

    t_segment_idxs = []
    for ts in t_segments:
        t_segment_idxs.append(int(ts / h_t))

    t_series = np.linspace(0, t_segments[-1], t_segment_idxs[-1])
    j_series = np.zeros(len(t_series))

    j_series[t_segment_idxs[0]:t_segment_idxs[1]] = j
    j_series[t_segment_idxs[2]:t_segment_idxs[3]] = -j
    j_series[t_segment_idxs[4]:t_segment_idxs[5]] = -j
    j_series[t_segment_idxs[6]:t_segment_idxs[7]] = j

    a_series = np.cumsum(j_series) * h_t
    v_series = np.cumsum(a_series) * h_t
    x_series = np.cumsum(v_series) * h_t
    
    return (t_series, j_series, a_series, v_series, x_series)

In [ ]:
def plot_trapezoid(
    t_series, j_series, a_series, v_series, x_series,
    t_jerk, t_a, t_u, a_target, v_target,
    x_target, v_max, a_max, j
):
    plot_step = 1
    if len(t_series) > 10000:
        plot_step = len(t_series) // 10000

    plt.figure(figsize=(10, 10))

    plt.subplot(4, 1, 1)
    plt.title("Координата")
    plt.plot(t_series[::plot_step], x_series[::plot_step])
    plt.axhline(x_target, linestyle="dashed", label="$x_{target}$")
    plt.legend()
    plt.xlabel("Время, с")
    plt.ylabel("Расстояние, м")

    plt.subplot(4, 1, 2)
    plt.title("Скорость")
    plt.plot(t_series[::plot_step], v_series[::plot_step])
    plt.axhline(v_target, linestyle="dashed", label="$v_{target}$")
    plt.axhline(v_max, linestyle="dashdot", label="$v_{max}$", color="C3")
    plt.legend()
    plt.xlabel("Время, с")
    plt.ylabel("Скорость, м/с")

    plt.subplot(4, 1, 3)
    plt.title("Ускорение")
    plt.plot(t_series[::plot_step], a_series[::plot_step])
    plt.axhline(a_target, linestyle="dashed", label="$a_{target}$")
    plt.axhline(a_max, linestyle="dashdot", label="$a_{max}$", color="C3")
    plt.legend()
    plt.xlabel("Время, с")
    plt.ylabel("Ускорение, м/с²")

    plt.subplot(4, 1, 4)
    plt.title("Рывок")
    plt.plot(t_series[::plot_step], j_series[::plot_step])
    plt.xlabel("Время, с")
    plt.ylabel("Рывок, м/с³")

    plt.tight_layout()
    plt.show()

## Интерактивный подбор параметров и построение трапеции

In [ ]:
x_target_input = widgets.BoundedFloatText(
    value=0.05,
    min=0.001,
    max=2.0,
    step=0.1,
    description="$x_{target}$, м",
    tooltip="Целевая позиция"
)
v_max_input = widgets.BoundedFloatText(
    value=0.1,
    min=0.001,
    max=100,
    step=0.1,
    description="$v_{max}$, м/с",
    tooltip="Абсолютная максимальная скорость (реальная максимальная может быть меньше)"
)
a_max_input = widgets.BoundedFloatText(
    value=4,
    min=0.01,
    max=1000,
    step=0.1,
    description="$a_{max}$, м/с²",
    tooltip="Абсолютное максимальное ускорение (реальное может быть меньше)"
)
j_input = widgets.BoundedFloatText(
    value=100000,
    min=0.001,
    max=10000000,
    step=0.1,
    description="$j$, м/с³",
    tooltip="Рывок"
)

calculate_trapezoid_button = widgets.Button(
    description="Рассчитать профиль",
    tooltip="Целевая позиция"
)

output = widgets.Output()

def calculate_and_plot_trapezoid(b):    
    with output:
        output.clear_output()
        x_target = float(x_target_input.value)
        v_max = float(v_max_input.value)
        a_max = float(a_max_input.value)
        j = float(j_input.value)
        
        t_jerk, t_a, t_u, a_target, v_target = calcaulate_timings_and_targets(x_target, v_max, a_max, j)
        t_series, j_series, a_series, v_series, x_series = calculate_time_series(t_jerk, t_a, t_u, j)
        plot_trapezoid(
            t_series, j_series, a_series, v_series, x_series,
            t_jerk, t_a, t_u, a_target, v_target,
            x_target, v_max, a_max, j
        )
        

calculate_trapezoid_button.on_click(calculate_and_plot_trapezoid)

display(
    x_target_input,
    v_max_input,
    a_max_input,
    j_input,
    calculate_trapezoid_button,
    output
)